In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pygam import LinearGAM, s
from sklearn.model_selection import train_test_split

In [10]:
df_2024 = pd.read_csv(
    "/Users/danishmak/Documents/Data Engineering/Final_Project_DE/archive/cbb23.csv"
)

In [11]:
df_train = pd.read_csv(
    "/Users/danishmak/Documents/Data Engineering/Final_Project_DE/archive/cbb22.csv"
)

In [12]:
df_train.columns

Index(['TEAM', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O',
       'EFGD_D', 'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D',
       '3P_O', '3P_D', 'ADJ_T', 'WAB', 'POSTSEASON', 'SEED'],
      dtype='object')

In [13]:
df_train["G"]

0      32
1      40
2      34
3      37
4      39
       ..
353    25
354    32
355    25
356    29
357    27
Name: G, Length: 358, dtype: int64

In [14]:
df_train = df_train.replace(np.nan, 0)
df_2024 = df_2024.replace(np.nan, 0)

# Extract relevant features and target variable from the 2023 data
X_train = df_train[
    [
        "G",
        "ADJOE",
        "ADJDE",
        "BARTHAG",
        "TOR",
        "TORD",
        "ORB",
        "DRB",
        "FTR",
        "FTRD",
        "2P_O",
        "2P_D",
        "3P_O",
        "3P_D",
        "ADJ_T",
        "WAB",
        "SEED",
    ]
]
y_train = df_train["W"]

# Build and fit the GAM model on 2023 data
gam = LinearGAM(
    s(0)
    + s(1)
    + s(2)
    + s(3)
    + s(4)
    + s(5)
    + s(6)
    + s(7)
    + s(8)
    + s(9)
    + s(10)
    + s(11)
    + s(12)
    + s(13)
    + s(14)
)
gam.fit(X_train, y_train)

# Prepare 2024 data and predict team wins
X_2024 = df_2024[
    [
        "G",
        "ADJOE",
        "ADJDE",
        "BARTHAG",
        "TOR",
        "TORD",
        "ORB",
        "DRB",
        "FTR",
        "FTRD",
        "2P_O",
        "2P_D",
        "3P_O",
        "3P_D",
        "ADJ_T",
        "WAB",
        "SEED",
    ]
]
teams_2024 = df_2024["TEAM"]  # Ensure TEAM column is present for ranking

# Predict on the 2024 data
y_pred_2024 = gam.predict(X_2024)

# Combine predictions with team names
results_2024 = pd.DataFrame({"TEAM": teams_2024, "Predicted_Wins": y_pred_2024})

# Sort by predicted wins in descending order
ranked_results_2023 = results_2024.sort_values(
    by="Predicted_Wins", ascending=False
).reset_index(drop=True)

# Print ranked teams with predicted wins for 2024
print(ranked_results_2023)

                 TEAM  Predicted_Wins
0    Florida Atlantic       34.999998
1             Houston       32.999999
2       San Diego St.       32.000000
3             Alabama       31.000000
4         Connecticut       31.000000
..                ...             ...
358          Hartford        5.000000
359        Louisville        4.000000
360        California        3.000000
361         Green Bay        3.000000
362      LIU Brooklyn        3.000000

[363 rows x 2 columns]


In [15]:
ranked_results_2023

,TEAM,Predicted_Wins
0,Florida Atlantic,34.999998
1,Houston,32.999999
2,San Diego St.,32.000000
3,Alabama,31.000000
4,Connecticut,31.000000
...,...,...
358,Hartford,5.000000
359,Louisville,4.000000
360,California,3.000000
361,Green Bay,3.000000
